This notebook provides predictions for housing prices in San Francisco, using XGBoost. The initial data analysis and feature engineering is provided in the HousingPrices notebook.

First we load the data and engineer the features as discussed in the previous notebook.

In [1]:
import xgboost as xgb

In [2]:
import pandas as pd
import numpy as np

data_key = 'housing_data_raw.csv'
housing = pd.read_csv(data_key)
base_housing = housing
df = pd.DataFrame(housing)

def drop_outliers(data):
    data = data.drop([1618])
    data = data.drop([3405])
    data = data.drop([10652])
    data = data.drop([954])
    data = data.drop([11136])
    data = data.drop([5103])
    data = data.drop([916])
    data = data.drop([10967])
    data = data.drop([7383])
    data = data.drop([1465])
    data = data.drop([8967])
    data = data.drop([8300])
    data = data.drop([4997])  
    return data

housing = drop_outliers(housing)
housing['finishedsqft'].sort_values()
housing['lastsolddateint'] = pd.to_datetime(housing['lastsolddate'], format='%m/%d/%Y').astype('int')
housing['lastsolddateint'] = housing['lastsolddateint']/1000000000
housing = housing[housing['lastsolddateint'].notnull()]
clean_data = housing.copy()
def drop_geog(data, keep = []):
    remove_list = ['info','address','z_address','longitude','latitude','neighborhood','lastsolddate','zipcode','zpid','usecode', 'zestimate','zindexvalue']
    for k in keep:
        remove_list.remove(k)
    data = data.drop(remove_list, axis=1)
    data = data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1)
    return data

housing = drop_geog(housing)

In [3]:
from sklearn.model_selection import train_test_split

def split_data(data):
    y = data['lastsoldprice']
    X = data.drop('lastsoldprice', axis=1)
    # Return (X_train, X_test, y_train, y_test)
    return train_test_split(X, y, test_size=0.3, random_state=30)

housing_split = split_data(housing)
#regression_model = lin_reg(*housing_split)
housing_cleaned = drop_geog(clean_data.copy(), ['neighborhood'])
one_hot = pd.get_dummies(housing_cleaned['neighborhood'])
housing_cleaned = housing_cleaned.drop('neighborhood',axis = 1)

from sklearn.preprocessing import StandardScaler
(X_train, X_test, y_train, y_test) = split_data(housing_cleaned)
scaler = StandardScaler()
scaler.fit(X_train)
X_train[X_train.columns] = scaler.transform(X_train[X_train.columns])
X_train = X_train.join(one_hot)
X_test[X_test.columns] = scaler.transform(X_test[X_test.columns])
X_test = X_test.join(one_hot)

(X_val, X_test, y_val, y_test) = train_test_split(X_test, y_test, test_size=0.5, random_state=30)

Next, we make some initial predictions on this data with the XGBoost library. Using GridSearchCV allows us to build the models using a number of possible hyperparameters, choosing the best set. This takes quite a while, so the best params (discovered by training the models) are left commented in case the reader prefers to use only those.

In [4]:
#data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)
from sklearn.model_selection import GridSearchCV
tuned_parameters = {'alpha': [0.001, 0.01, 0.1], 
                    'learning_rate': [.01, .3, .5, 1],
                   'max_depth': [2, 6,  10],
                  'n_estimators': [2, 10, 100, 200],
                   'colsample_bytree': [.1, .3, .9, 1],
                   'objective': ['reg:linear']}

#tuned_parameters = {'alpha': [.01], 
#                    'learning_rate': [.3],
#                   'max_depth': [6],
#                    'n_estimators': [100],
#                   'colsample_bytree': [.9],
#                   'objective': ['reg:linear']}


xg_reg = GridSearchCV(xgb.XGBRegressor(), tuned_parameters, cv=5, n_jobs=-1, verbose=1)

#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                #max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done 215 tasks      | elapsed:   49.7s
[Parallel(n_jobs=-1)]: Done 465 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 815 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done 1265 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done 1815 tasks      | elapsed: 19.9min
[Parallel(n_jobs=-1)]: Done 2465 tasks      | elapsed: 25.2min
[Parallel(n_jobs=-1)]: Done 2880 out of 2880 | elapsed: 31.6min finished


In [5]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def mean_abs_perc_error(y_test, preds):
    total = 0
    n = len(y_test)
    for i in range(0, n-1):
        diff = abs(preds[i] - y_test.iloc[i]) / y_test.iloc[i]
        total = total + diff
    return total / n

mae = mean_absolute_error(y_test, preds)
mape = mean_abs_perc_error(y_test, preds)
rmse = np.sqrt(mean_squared_error(y_test, preds))
r2 = r2_score(y_test, preds)
print("RMSE: %f" % (rmse))
print("MAE: %f" % (mae))
print("MAPE: %f" % (mape))
print("R^2: %f" % (r2))

RMSE: 490845.120008
MAE: 260408.456212
MAPE: 1.288777
R^2: 0.763546


In [6]:
xg_reg.best_params_

{'alpha': 0.001,
 'colsample_bytree': 0.9,
 'learning_rate': 0.3,
 'max_depth': 6,
 'n_estimators': 100,
 'objective': 'reg:linear'}

We can see that, as many have argued, XGBoost gives us the best results, a fair amount better than we saw in the other notebooks. Now, let's see our results when we train only on pre-2016 in order to predict 2016 prices.

In [7]:
housing_cleaned = drop_geog(clean_data.copy(), ['neighborhood'])
one_hot = pd.get_dummies(housing_cleaned['neighborhood'])
housing_cleaned = housing_cleaned.drop('neighborhood',axis = 1)

housing_cleaned_2016 = housing_cleaned[housing_cleaned['lastsolddateint'] >= 1451606400.0]
housing_cleaned_pre2016 = housing_cleaned[housing_cleaned['lastsolddateint'] < 1451606400.0]

X_train = housing_cleaned_pre2016.drop('lastsoldprice',axis = 1)
y_train = housing_cleaned_pre2016['lastsoldprice']
X_test = housing_cleaned_2016.drop('lastsoldprice',axis = 1)
y_test = housing_cleaned_2016['lastsoldprice']

scaler = StandardScaler()
scaler.fit(X_train)
X_train[X_train.columns] = scaler.transform(X_train[X_train.columns])
X_train = X_train.join(one_hot)
X_test[X_test.columns] = scaler.transform(X_test[X_test.columns])
X_test = X_test.join(one_hot)

(X_val, X_test, y_val, y_test) = train_test_split(X_test, y_test, test_size=0.5, random_state=30)

In [8]:
xg_reg = GridSearchCV(xgb.XGBRegressor(), tuned_parameters, cv=5, n_jobs=-1, verbose=1)

#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                #max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
mae = mean_absolute_error(y_test, preds)
mape = mean_abs_perc_error(y_test, preds)
rmse = np.sqrt(mean_squared_error(y_test, preds))
r2 = r2_score(y_test, preds)
print("RMSE: %f" % (rmse))
print("MAE: %f" % (mae))
print("MAPE: %f" % (mape))
print("R^2: %f" % (r2))

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 215 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 465 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 815 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 1265 tasks      | elapsed: 15.1min
[Parallel(n_jobs=-1)]: Done 1815 tasks      | elapsed: 24.2min
[Parallel(n_jobs=-1)]: Done 2465 tasks      | elapsed: 31.4min
[Parallel(n_jobs=-1)]: Done 2880 out of 2880 | elapsed: 40.1min finished


RMSE: 647964.742586
MAE: 313178.272478
MAPE: 0.246647
R^2: 0.597122


In [9]:
xg_reg.best_params_

{'alpha': 0.001,
 'colsample_bytree': 1,
 'learning_rate': 0.3,
 'max_depth': 6,
 'n_estimators': 100,
 'objective': 'reg:linear'}

Unsurprisingly, these results are worse than the previous run, but still better than we have seen with many of our other algorithms, equal to our best result (KNN) in the other set of algorithms in the HousePrices notebook.

Now, let's look at training only on post-recession prices to predict 2016 prices.

In [10]:
housing_cleaned = drop_geog(clean_data.copy(), ['neighborhood'])
one_hot = pd.get_dummies(housing_cleaned['neighborhood'])
housing_cleaned = housing_cleaned.drop('neighborhood',axis = 1)

housing_cleaned_2016 = housing_cleaned[housing_cleaned['lastsolddateint'] >= 1451606400.0]
housing_cleaned_pre2016 = housing_cleaned[housing_cleaned['lastsolddateint'] < 1451606400.0]
housing_cleaned_2012_2015 = housing_cleaned_pre2016[housing_cleaned_pre2016['lastsolddateint'] >= 1325376000.0]

X_train = housing_cleaned_2012_2015.drop('lastsoldprice',axis = 1)
y_train = housing_cleaned_2012_2015['lastsoldprice']
X_test = housing_cleaned_2016.drop('lastsoldprice',axis = 1)
y_test = housing_cleaned_2016['lastsoldprice']

scaler = StandardScaler()
scaler.fit(X_train)
X_train[X_train.columns] = scaler.transform(X_train[X_train.columns])
X_train = X_train.join(one_hot)
X_test[X_test.columns] = scaler.transform(X_test[X_test.columns])
X_test = X_test.join(one_hot)

(X_val, X_test, y_val, y_test) = train_test_split(X_test, y_test, test_size=0.5, random_state=30)

In [11]:
xg_reg = GridSearchCV(xgb.XGBRegressor(), tuned_parameters, cv=5, n_jobs=-1, verbose=1)

#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                #max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
mae = mean_absolute_error(y_test, preds)
mape = mean_abs_perc_error(y_test, preds)
rmse = np.sqrt(mean_squared_error(y_test, preds))
r2 = r2_score(y_test, preds)
print("RMSE: %f" % (rmse))
print("MAE: %f" % (mae))
print("MAPE: %f" % (mape))
print("R^2: %f" % (r2))

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 215 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 465 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 815 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 1265 tasks      | elapsed: 14.9min
[Parallel(n_jobs=-1)]: Done 1815 tasks      | elapsed: 23.8min
[Parallel(n_jobs=-1)]: Done 2465 tasks      | elapsed: 30.8min
[Parallel(n_jobs=-1)]: Done 2880 out of 2880 | elapsed: 39.3min finished


RMSE: 652074.291434
MAE: 322219.748081
MAPE: 0.247582
R^2: 0.591996


In [12]:
xg_reg.best_params_

{'alpha': 0.001,
 'colsample_bytree': 0.9,
 'learning_rate': 0.3,
 'max_depth': 6,
 'n_estimators': 100,
 'objective': 'reg:linear'}

Finally, though this result is better than most, it is not quite as good (but not substantially worse) than RandomForest and GBM.